In [ ]:
# C4. Créer une base de données dans le respect du RGPD en élaborant les modèles conceptuels et physiques des données à partir des données préparées
# et en programmant leur import afin de stocker le jeu de données du projet.

In [ ]:
#1. Récapitulatif des points à valider :
2# Modèles conceptuels et physiques bien conçus (Merise).
3# Modèle physique fonctionnel sans erreurs lors de l'implémentation.
4# Choix de la base de données justifié.
5# Respect du RGPD lors de la manipulation des données.
6# Procédures d'installation et de configuration documentées.
7# Script d'import fonctionnel avec gestion des relations et des conflits.
8# Documentation versionnée du script dans un dépôt Git

In [ ]:
# ### Langages et dépendances ( a inclure dans la documentation)
# - **Langage** : Python 3.x
# - **Dépendances** :
#  - `psycopg2` : Bibliothèque pour se connecter à PostgreSQL.
#  - `pandas` : Bibliothèque pour la manipulation des fichiers CSV et des DataFrames.

### Installation des dépendances
# Pour installer les dépendances, exécuter la commande suivante :
# ```bash
# pip install psycopg2 pandas 

In [ ]:
### Registre des traitements de données personnelles
# - **Données traitées** : noms des auteurs (colonne `authors` dans la table `Auteur`).
# - **Finalité** : stockage des noms des auteurs des livres pour le projet de recommandations de lectures.
# - **Localisation** : les données sont stockées dans la base de données PostgreSQL.
# - **Accès** : seuls les administrateurs de la base de données ont accès aux données personnelles via des accès sécurisés.
# - **Sécurisation** : les accès sont protégés par des mots de passe forts et les données sont stockées dans un environnement sécurisé.
# - **Durée de conservation** : les données personnelles des auteurs seront conservées tant que le service est opérationnel, sauf demande de suppression spécifique d'un auteur.
# - **Procédures de suppression** : possibilité de suppression manuelle des données personnelles à la demande, ou suppression automatique après X années d'inactivité.

In [ ]:
### Procédures de tri des données personnelles
# - **Objectif** : Assurer la conformité avec le RGPD en identifiant et supprimant les données personnelles inutiles.
# - **Données concernées** : noms des auteurs (colonne `authors`).
# - **Fréquence** : les données sont revues tous les 6 mois pour vérifier si elles sont encore nécessaires.
# - **Processus de suppression** :
#   - Si un auteur fait une demande de suppression, ses données seront supprimées manuellement par l'administrateur dans les 30 jours suivant la demande.
#   - Si les données sont jugées non nécessaires (par exemple, pour les auteurs décédés ou pour des livres retirés), elles seront anonymisées ou supprimées.
# - **Anonymisation** : remplacer les noms d'auteurs par des identifiants anonymes si cela est nécessaire.


In [ ]:
# On a créé 5 tables au total dans PGadmin pour les différentes liaisons entre les données etc , tout est dans 'suivi du projet final' word ,il reste
# plus qu'à envoyer dans pgadmin cette version finale

In [1]:
%pip install psycopg2

In [ ]:
# Fichier nettoyé final path : C:\Users\User\Downloads\CoursAlternance\Chefoeuvre\RecommendationsLectures\final_dataset_clean.csv

In [10]:
import psycopg2
import pandas as pd

# Connexion à PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="RecommendationsLectures",
    user="postgres",
    password="Lrk389229!"
)
cur = conn.cursor()

# Charger le dataset final nettoyé
df_books = pd.read_csv(r"C:\Users\User\Downloads\CoursAlternance\Chefoeuvre\RecommendationsLectures\final_dataset_clean.csv")

# Importer les livres
for index, row in df_books.iterrows():
    cur.execute("""
        INSERT INTO Livre (title, description, published_year, average_rating)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (title) DO NOTHING;
    """, (row['title'], row['description'], row['published_year'], row['average_rating']))

# Importer les auteurs
for index, row in df_books.iterrows():
    authors = row['authors'].split(';')  # Séparer les auteurs si plusieurs
    for author in authors:
        cur.execute("""
            INSERT INTO Auteur (author_name)
            VALUES (%s)
            ON CONFLICT (author_name) DO NOTHING;
        """, (author,))

# Importer les catégories
for index, row in df_books.iterrows():
    categories = row['categories'].split(';')  # Séparer les catégories si plusieurs
    for category in categories:
        cur.execute("""
            INSERT INTO Categorie (category_name)
            VALUES (%s)
            ON CONFLICT (category_name) DO NOTHING;
        """, (category,))

# Importer les associations Livre_Auteur
for index, row in df_books.iterrows():
    authors = row['authors'].split(';')
    for author in authors:
        cur.execute("""
            INSERT INTO Livre_Auteur (id_livre, id_auteur)
            SELECT l.id_livre, a.id_auteur
            FROM Livre l, Auteur a
            WHERE l.title = %s AND a.author_name = %s
            ON CONFLICT (id_livre, id_auteur) DO NOTHING;
        """, (row['title'], author))

# Importer les associations Livre_Categorie
for index, row in df_books.iterrows():
    categories = row['categories'].split(';')
    for category in categories:
        cur.execute("""
            INSERT INTO Livre_Categorie (id_livre, id_categorie)
            SELECT l.id_livre, c.id_categorie
            FROM Livre l, Categorie c
            WHERE l.title = %s AND c.category_name = %s
            ON CONFLICT (id_livre, id_categorie) DO NOTHING;
        """, (row['title'], category))

# Commit des transactions et fermer la connexion
conn.commit()
cur.close()
conn.close()
